In [1]:
from PDB_Order_Fixer import PDB_Order_Fixer
import mdtraj as md
import os
import numpy as np
import h5py

import datetime
import glob
import copy
from functools import partial 
import operator
import time

import random 
import subprocess
from subprocess import Popen
import sys
from io_functions import *
from custom_clusterer import *
from custom_tica import *
from custom_featurizer import *
from pdb_editing import *
from analysis import *
from io_functions import *
#from topology_fixing import *
from subsampling import *
from conversions import *
from custom_msm import *
from grids import *

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt

import matplotlib as mpl
from seaborn.apionly import set_palette
from IPython.display import set_matplotlib_formats

# configure plotting
set_matplotlib_formats('pdf', 'svg')
set_palette('Set1', n_colors=15, desat=None)

/home/enf/software/anaconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:66: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  inline backend."""
/home/enf/software/anaconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:71: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  'retina', 'jpeg', 'svg', 'pdf'.""")
/home/enf/software/anaconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:85: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  use `figure_formats` instead)""")
/home/enf/software/anaconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:95: DeprecationWarning: metadata {'config': True} wa

In this iPython notebook, we will featurize MOR ligand binding simulation by pairwise distances between the ligand and different receptor residues. We will then perform tICA and prospectively build an MSM. 

In [3]:
from detect_intermediates import *
from interpret_tICs import *

we are operating on biox3


In [4]:
from mor_h8_feature_types import *
from get_variable_names import *
from mor_h8_tica_config import *
from residue import Residue, Atom

tm6_tm3_residues
[R279, R165]
[65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 273, 274, 275, 276, 277, 278,

In [5]:
feature_name = "all_residues_4dkl_5c1m_under_cutoff6A_top-features-gamma-pdf"

(active_ref_dir, inactive_ref_dir, simulation_ref_dir, scripts_dir,
          ligand_dir, agonist_dir, inverse_agonist_dir, biased_agonist_dir, ref_receptors_dir, whole_trajectory_pnas,
          sasa_file) = get_base_files(base)

tica_dir = get_tica_dir(base, is_sparse, lag_time, n_components, feature_name, 
                                 wolf_string, shrinkage_string, rho_string)
ori_tica_dir = copy.deepcopy(tica_dir)
features_dir = get_features_dir(base, feature_name)

landmarks_dir = get_landmarks_dir(tica_dir)
analysis_dir = get_analysis_dir(tica_dir, n_clusters, sampling_method)
gmm_dir = get_gmm_dir(tica_dir)
rf_dir = get_rf_dir(tica_dir)


ref_tica_dir, ref_tica_coords = get_ref_tica_dirs(tica_dir)

graph_file = get_graph_file(tica_dir, msm_lag_time, n_clusters)

pnas_titles =  ["tm6_tm3_dist", "rmsd_npxxy_inactive", "rmsd_npxxy_active", "rmsd_connector_inactive", "rmsd_connector_active"]
pnas_features_dir = analysis_dir


(clusterer_dir, msm_model_dir, macrostate_dir, features_known, model_dir, projected_features_dir,
         projection_operator_dir, ktica_fit_model_filename, ktica_projected_data_filename, nystroem_data_filename,
         mutual_information_csv, pearson_csv) = get_tica_files(base, tica_dir, n_clusters, msm_lag_time, n_macrostates)

(standardized_features_dir, feature_residues_csv, feature_residues_pkl,
          contact_csv, ref_features_dir) = get_feature_files(features_dir)

(kmeans_csv, tica_coords_csv, features_csv, active_rmsd_dir, inactive_rmsd_dir, active_pnas_dir, inactive_pnas_joined, active_pnas_joined,
        clusters_map_file, ktica_clusters_map_file, analysis_file, combined_file, docking_summary, docking_joined, docking_z_scores_csv,
        aggregate_docking, aggregate_docking_joined, docking_pnas_joined, aggregate_docking_pnas, aggregate_docking_pnas_joined, docking_multiple_ligands,
        docking_distances_file, docking_pdf, mmgbsa_docking_distances, pnas_coords, mmgbsa_dir, mmgbsa_csv, mmgbsa_pdf, aggregate_mmgbsa,
        aggregate_mmgbsa_joined, aggregate_mmgbsa_pnas_joined, mmgbsa_z_scores_csv, active_clusters_csv, intermediate_clusters_csv,
        inactive_clusters_csv, pnas_clusters_averages, tica_clusters_averages, tica_classes_csv, tica_samples_csv, subgraph_save_base,
        degree_save_base, degree_map_csv, degree_z_map_csv, aggregate_docking_pnas_degree_z_joined, tic_residue_csv, feature_coefs_csv,
        duplicated_feature_coefs_csv) = get_analysis_files(analysis_dir, n_clusters, tica_dir, tica_dir, sampling_method, n_samples, precision,
                                                           msm_lag_time)

(inactive_pnas_distances_dir, active_pnas_distances_dir, active_pnas_all_distances_dir,
          inactive_pnas_distances_new_csv, active_pnas_distances_new_csv, active_pnas_joined, active_pnas_means, pnas_coords_dir,
          pnas_coords_csv, pnas_all_coords_csv, pnas_coords_hexbin_dir, pnas_coords_co_crystallized_docking_dir,
          pnas_coords_active_colors_dir, user_defined_features_file, reaction_coordinates_trajs_file) = get_pnas_files(whole_trajectory_pnas, pnas_features_dir)

features_dir = get_features_dir(base, feature_name)



graph_file = get_graph_file(tica_dir, msm_lag_time, n_clusters)
(scripts_dir, pymol_fixpdb_dir) = get_script_dir(scripts_dir)
(save_dir, reimaged_dir, mae_dir, combined_reimaged_dir, grid_dir, docking_dir) = get_docking_dirs(tica_dir, n_clusters, n_components, n_samples, sampling_method, precision)


/home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A_top-features-gamma-pdf
/home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A_top-features-gamma-pdf


In [6]:
print feature_residues_pkl

/home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A_top-features-gamma-pdf/feature_residues.pkl


In [7]:
from ipyparallel import Client
rc = Client()
print(len(rc.ids))
dview = rc[:]
dview.map(os.chdir, ['/home/enf/b2ar_analysis/conformation']*len(rc.ids))

/home/enf/software/anaconda/lib/python2.7/site-packages/ipyparallel/client/client.py:312: DeprecationWarning: Client._profile_default is deprecated: use @default decorator instead.
  def _profile_default(self):
/home/enf/software/anaconda/lib/python2.7/site-packages/ipyparallel/client/client.py:312: DeprecationWarning: Client._profile_default is deprecated: use @default decorator instead.
  def _profile_default(self):
/home/enf/software/anaconda/lib/python2.7/site-packages/IPython/core/profiledir.py:57: DeprecationWarning: ProfileDir._location_changed is deprecated: use @observe and @unobserve instead.
  def _location_changed(self, name, old, new):
/home/enf/software/anaconda/lib/python2.7/site-packages/IPython/core/profiledir.py:126: DeprecationWarning: ProfileDir._security_dir_changed is deprecated: use @observe and @unobserve instead.
  def _security_dir_changed(self, name, old, new):
/home/enf/software/anaconda/lib/python2.7/site-packages/IPython/core/profiledir.py:71: DeprecationW

350


/home/enf/software/anaconda/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):


<AsyncMapResult: chdir>

In [18]:
f = load_file("/home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_85.dataset")
print(np.shape(f))

loading /home/enf/md_simulations/MOR/h8_reimaged/featuresall_residues_4dkl_5c1m_under_cutoff6A-CA/apo_rep_85.dataset
(462, 4440)


In [21]:
print(f[:,0])
print(f[:,2221])

[ 0.64961469  0.66602194  0.66592467  0.63948542  0.63587338  0.60345757
  0.64630765  0.63133705  0.62835622  0.61737096  0.63898677  0.59233195
  0.61077648  0.62222731  0.66184503  0.62608761  0.60952699  0.58673173
  0.59007311  0.60639507  0.6193912   0.63611972  0.61656678  0.64260018
  0.64610952  0.65614671  0.63218528  0.62076133  0.59317166  0.62671173
  0.59976476  0.65676868  0.632653    0.56492603  0.66460592  0.61706424
  0.60110283  0.59100425  0.64788824  0.5579735   0.56510991  0.57512546
  0.62548161  0.61732984  0.60033673  0.51456451  0.54886299  0.54760832
  0.61654752  0.64133185  0.6113646   0.57381809  0.60062361  0.68652225
  0.69681996  0.56393993  0.6368382   0.65521145  0.60553265  0.60577291
  0.65196663  0.60596538  0.61692148  0.63084573  0.56684172  0.65449965
  0.55836672  0.59040231  0.58052677  0.61316198  0.61831766  0.57411927
  0.58879858  0.55381835  0.62335193  0.5448125   0.58469957  0.57175303
  0.53552169  0.58577573  0.56942433  0.56810707  0

In [ ]:
feature_name_residues_dict

In [ ]:
compute_user_defined_features_wrapper(traj_dir, traj_ext, inactive_dir, active_dir, structure,
                                          feature_name_residues_dict, user_defined_features_file, worker_pool = dview)
plot_columns(whole_trajectory_pnas, user_defined_features_file, titles = sorted(feature_name_residues_dict.keys()), tICA=False, scale=1.0, refcoords_file=None)

In [ ]:
ref_user_defined_features_file = "%s/ref_coords.h5" %whole_trajectory_pnas
compute_user_defined_features_wrapper(ref_receptors_dir, ".pdb", inactive_dir, active_dir, structure,
                                          feature_name_residues_dict, ref_user_defined_features_file, worker_pool = None)

In [ ]:
import analysis
reload(analysis)
from analysis import *
plot_columns(whole_trajectory_pnas, user_defined_features_file, titles = sorted(feature_name_residues_dict.keys()), tICA=False, scale=1.0, refcoords_file=ref_user_defined_features_file)

In [ ]:
whole_trajectory_pnas

In [13]:
import custom_featurizer
reload(custom_featurizer)
from custom_featurizer import *

In [16]:
#coords_bounds_dict = {'rmsd_npxxy_inactive': [0, 0.5, 1.0], 'tm6_tm3_dist': [6.0, 8.0, 15.0]}
coords_bounds_dict = {'rmsd_npxxy_inactive': [0, 0.5, 1.0], 'tm6_tm3_packing': [8.0, 9.5, 15.0]}

traversing_trajs = reaction_coordinate_sampler(traj_dir, traj_ext, user_defined_features_file, 
                            feature_name_residues_dict, coords_bounds_dict, reaction_coordinates_trajs_file)


loading "/home/enf/md_simulations/MOR/h8_reimaged/all_pnas_features/user_defined_features.h5"...
(359837, 10)
Analyzing tm6_tm3_packing
Analyzing rmsd_npxxy_inactive
{'rmsd_npxxy_inactive': ['apo_rep_100.h5', 'apo_rep_103.h5', 'apo_rep_106.h5', 'apo_rep_109.h5', 'apo_rep_122.h5', 'apo_rep_124.h5', 'apo_rep_131.h5', 'apo_rep_133.h5', 'apo_rep_143.h5', 'apo_rep_145.h5', 'apo_rep_146.h5', 'apo_rep_149.h5', 'apo_rep_15.h5', 'apo_rep_152.h5', 'apo_rep_155.h5', 'apo_rep_163.h5', 'apo_rep_169.h5', 'apo_rep_170.h5', 'apo_rep_184.h5', 'apo_rep_185.h5', 'apo_rep_190.h5', 'apo_rep_197.h5', 'apo_rep_199.h5', 'apo_rep_210.h5', 'apo_rep_212.h5', 'apo_rep_230.h5', 'apo_rep_231.h5', 'apo_rep_234.h5', 'apo_rep_25.h5', 'apo_rep_255.h5', 'apo_rep_31.h5', 'apo_rep_34.h5', 'apo_rep_35.h5', 'apo_rep_37.h5', 'apo_rep_4.h5', 'apo_rep_41.h5', 'apo_rep_43.h5', 'apo_rep_45.h5', 'apo_rep_5.h5', 'apo_rep_57.h5', 'apo_rep_63.h5', 'apo_rep_65.h5', 'apo_rep_68.h5', 'apo_rep_7.h5', 'apo_rep_73.h5', 'apo_rep_76.h5', 'a

In [17]:
intersection_trajs = [a for a in traversing_trajs['rmsd_npxxy_inactive'] if a in traversing_trajs['tm6_tm3_packing']]
intersection_trajs

['apo_rep_109.h5',
 'apo_rep_15.h5',
 'apo_rep_35.h5',
 'apo_rep_68.h5',
 'bu72_reimaged_rep_102.h5',
 'bu72_reimaged_rep_53.h5']

In [ ]:
import custom_featurizer
reload(custom_featurizer)
from custom_featurizer import *
featurize_contacts_custom(traj_dir, features_dir = features_dir, traj_ext = traj_ext, contact_residue_pairs_file = feature_residues_pkl, structures=[inactive_dir, active_dir], contact_residues=contact_residues,
                          residues_map = None, contact_cutoff = cutoff, parallel = True, exacycle = exacycle, traj_top_structure = None, iterative=False,
                          user_specified_atom_residue_pairs = [], load_from_file=True)

In [ ]:
import pickle
with open(feature_residues_pkl, "rb") as f:
    feature_residues = pickle.load(f)
print(feature_residues[0])
print(feature_residues[0][0].__dict__)
print(feature_residues[0][1].__dict__)
print(feature_residues[727])

In [ ]:
fit_and_transform(features_directory = features_dir, model_dir = tica_dir, stride=5, lag_time = lag_time, n_components = n_components, sparse = sparse, wolf = wolf, rho = rho, shrinkage = shrinkage, parallel=True, traj_ext = traj_ext)

In [ ]:
from imp import reload
import analysis
reload(analysis)
from analysis import *
plot_columns(tica_dir, projected_features_dir, titles = ["tIC%d" %j for j in range(1,11)], tICA = True, scale = 1.0, refcoords_file = None)

In [ ]:
import interpret_tICs
reload(interpret_tICs)
from interpret_tICs import *
tic_components_dir = tica_dir
interpret_tIC_components(projection_operator_dir, tic_components_dir, feature_residues_pkl, n_tica_components=n_components, percentile=95)


In [ ]:
plot_columns(pnas_features_dir, user_defined_features_file, titles = feature_name_residues_dict.keys(), tICA=False, scale=1.0, refcoords_file=None)

In [ ]:
"""
New tIC interpretation function: 

1. Normalize all features (separate function, useful for learning in general.)
2. For each feature: discretize into 1's and 0's based on above or below zero z-score
3. Fit feature/tIC combo: fit logistic regression CV. 
4. For each tIC: rank the features by how "good/strong" fit is.
    b. save the tIC value at which the switch occurs. 


"""

In [ ]:
import custom_featurizer
reload(custom_featurizer)
from custom_featurizer import *

import mor_h8_feature_types
reload(mor_h8_feature_types)
from mor_h8_feature_types import *

print feature_name_residues_dict
print structure

compute_user_defined_features_wrapper(traj_dir, traj_ext, inactive_dir, active_dir, structure,
                                          feature_name_residues_dict, user_defined_features_file)
plot_columns(pnas_features_dir, user_defined_features_file, titles = feature_name_residues_dict.keys(), tICA=False, scale=1.0, refcoords_file=None)

In [ ]:
plot_pnas_vs_tics(user_defined_features_file, projected_features_dir, sorted(feature_name_residues_dict.keys()), tica_dir)

In [ ]:
bu72_trajs = []
apo_trajs = []
for i, traj in enumerate(get_trajectory_files(traj_dir, traj_ext)):
    if "bu72" in traj and "reimaged" not in traj:
        bu72_trajs.append(i)
    elif "apo" in traj:
        apo_trajs.append(i)

user_defined_coords = load_file(user_defined_features_file)

            

In [ ]:
bu72_pnas = [user_defined_coords[i] for i in bu72_trajs]
apo_pnas = [user_defined_coords[i] for i in apo_trajs]

In [ ]:
from imp import reload
import analysis
reload(analysis)
from analysis import *
import seaborn as sns
bu72_pnas_concatenated = np.concatenate(bu72_pnas)
print(np.shape(bu72_pnas_concatenated))
apo_pnas_concatenated = np.concatenate(apo_pnas)
bu72_pnas_file = "%s/bu72.h5" % whole_trajectory_pnas
apo_pnas_file = "%s/apo.h5" % whole_trajectory_pnas
verbosedump(bu72_pnas, bu72_pnas_file)
verbosedump(apo_pnas, apo_pnas_file)

columns=sorted(feature_name_residues_dict.keys())
plot_columns(whole_trajectory_pnas, bu72_pnas_file, titles=columns, main = "BU72", tICA = False, scale = 1.0, refcoords_file = None)
plot_columns(whole_trajectory_pnas, apo_pnas_file, titles=columns, main = "Apo", tICA = False, scale = 1.0, refcoords_file = None)

#df = pd.DataFrame(bu72_pnas_concatenated, columns=sorted(feature_name_residues_dict.keys()))
#sns.jointplot(x="tm6_tm3_dist", y="rmsd_npxxy_inactive", data=df, kind="kde")
#plt.hexbin(bu72_pnas_concatenated[:,0], bu72_pnas_concatenated[:,2], cmap=plt.cm.YlOrRd_r)

In [ ]:
tica_coords = verboseload(projected_features_dir)
bu72_tica = [tica_coords[i] for i in bu72_trajs]
apo_tica = [tica_coords[i] for i in apo_trajs]
bu72_tica_file = "%s/bu72_tica.h5" % tica_dir
apo_tica_file = "%s/apo_tica.h5" % tica_dir
verbosedump(bu72_tica, bu72_tica_file)
verbosedump(apo_tica, apo_tica_file)

In [ ]:
columns = ["tIC%d" %i for i in range(1,n_components+1)]
plot_columns(tica_dir, bu72_tica_file, titles=columns, main = "BU72", tICA = False, scale = 1.0, refcoords_file = None)
plot_columns(tica_dir, apo_tica_file, titles=columns, main = "Apo", tICA = False, scale = 1.0, refcoords_file = None)



In [ ]:
clusterer_dir = os.path.join(tica_dir, "apo_original_300_clusters.h5")

In [ ]:
cluster_minikmeans(tica_dir, apo_tica_file, traj_dir, n_clusters=n_clusters, clusterer_dir=clusterer_dir)

In [ ]:
import custom_clusterer
from imp import reload
reload(custom_clusterer)
from custom_clusterer import *
reseed_from_clusterer(clusterer_dir, "apo_original", tica_dir, apo_tica_file, [get_trajectory_files(traj_dir, traj_ext)[i] for i in apo_trajs])

In [ ]:
clusterer = verboseload(clusterer_dir)
cluster_labels = clusterer.labels_
bu72_clusters = [cluster_labels[i] for i in bu72_trajs]
apo_clusters = [cluster_labels[i] for i in apo_trajs]
bu72_clusterer = copy.deepcopy(clusterer)
bu72_clusterer.labels_ = bu72_clusters
bu72_clusterer_file = "%s/bu72_clusterer.h5" %tica_dir
verbosedump(bu72_clusterer, bu72_clusterer_file)

apo_clusterer = copy.deepcopy(clusterer)
apo_clusterer.labels_ = apo_clusters
apo_clusterer_file = "%s/apo_clusterer.h5" %tica_dir
verbosedump(apo_clusterer, apo_clusterer_file)


In [ ]:
from imp import reload
import analysis
reload(analysis)
from analysis import *
bu72_clusters_map = make_clusters_map(verboseload(bu72_clusterer_file))
count_tuples = []
for i in range(0,n_clusters):
    count_tuples.append((i, len(bu72_clusters_map[i])))
count_tuples.sort(key=operator.itemgetter(1))
min_populated_clusters = [count_tuples[i][0] for i in range(0,16)]
print(min_populated_clusters)
plot_all_tics_and_clusters(tica_dir, projected_features_dir, bu72_clusterer_file, lag_time, tic_range = [0], main="BU72", label = "cluster_id", active_cluster_ids = min_populated_clusters)

In [ ]:
count_tuples

In [ ]:
apo_clusters_map = make_clusters_map(verboseload(apo_clusterer_file))
count_tuples = []
for i in range(0,n_clusters):
    count_tuples.append((i, len(apo_clusters_map[i])))
count_tuples.sort(key=operator.itemgetter(1))
min_populated_clusters = [count_tuples[i][0] for i in range(0,16)]
print(min_populated_clusters)
plot_all_tics_and_clusters(tica_dir, projected_features_dir, apo_clusterer_file, lag_time, tic_range = [0], main="Apo", label = "cluster_id", active_cluster_ids = min_populated_clusters)

In [ ]:
traj_index_frame_pairs = list(find_closest_indices_to_cluster_center(bu72_tica_file, bu72_clusterer_file))
traj_index_frame_pairs = [tuple(pair) for pair in traj_index_frame_pairs]
traj_files = get_trajectory_files(traj_dir)
bu72_traj_files = [traj_files[i] for i in bu72_trajs]
print(traj_index_frame_pairs)

for i, traj_index_frame_pair in enumerate(traj_index_frame_pairs):
    traj_index, frame = traj_index_frame_pair
    if i in min_populated_clusters:
        print("Looking at cluster %d" % i)
        snapshot = md.load_frame(bu72_traj_files[traj_index], index=frame)
        snapshot.save("%s/mincount_snapshot_cluster%d.rst7" % (tica_dir, i))
        snapshot.save("%s/mincount_snapshot_cluster%d.pdb" % (tica_dir, i))

In [ ]:
traj_index_frame_pairs = list(find_closest_indices_to_cluster_center(apo_tica_file, apo_clusterer_file))
traj_index_frame_pairs = [tuple(pair) for pair in traj_index_frame_pairs]
traj_files = get_trajectory_files(traj_dir)
apo_traj_files = [traj_files[i] for i in apo_trajs]
print(traj_index_frame_pairs)

for i, traj_index_frame_pair in enumerate(traj_index_frame_pairs):
    traj_index, frame = traj_index_frame_pair
    if i in min_populated_clusters:
        print("Looking at cluster %d" % i)
        snapshot = md.load_frame(apo_traj_files[traj_index], index=frame)
        snapshot.save("%s/apo_mincount_snapshot_cluster%d.rst7" % (tica_dir, i))
        snapshot.save("%s/apo_mincount_snapshot_cluster%d.pdb" % (tica_dir, i))

In [ ]:
import custom_msm
reload(custom_msm)
from custom_msm import *
plot_timescales(bu72_clusterer_file, n_clusters, tica_dir, main="BU72", lag_times=range(1,12,2))
plot_timescales(apo_clusterer_file, n_clusters, tica_dir, main="Apo", lag_times=range(1,12,2))

In [ ]:
build_msm(bu72_clusterer_file, lag_time, msm_model_dir)
build_msm(apo_clusterer_file, lag_time, msm_model_dir)

In [ ]:
plot_all_tics_and_clusters(tica_dir, projected_features_dir, clusterer_dir, lag_time, label = "cluster_id", active_cluster_ids = range(0,n_clusters), intermediate_cluster_ids = [], inactive_cluster_ids = [])

In [ ]:
plot_timescales(clusterer_dir, n_clusters, tica_dir, list(range(1,25)))


In [ ]:
lag_time = 5
msm_model_dir = "%s/msm_lag_time%d.h5" % (tica_dir, lag_time)
#build_msm(clusterer_dir, lag_time=lag_time, msm_model_dir=msm_model_dir)


In [ ]:
graph_file = "%s/msm_lag_time%d_graph.graphml" % (tica_dir, lag_time)
construct_graph(msm_model_dir, clusterer_dir, n_clusters, 5, 5, graph_file, inactive = None, active = None, pnas_clusters_averages = None, tica_clusters_averages = None, docking=None, macrostate = None)


In [ ]:
from imp import reload
import custom_msm
reload(custom_msm)
from custom_msm import *
msm_file = msm_model_dir
sampled_frames_file = '/home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/tica_ligand_protein_contacts_3_sparse_0pt0025/sampled_frames_clusters50_samples1.h5'
msm_trajectory_filename = '/home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/tica_ligand_protein_contacts_3_sparse_0pt0025/msm_lagtime5_start48_3'
make_msm_trajectory(msm_file, projected_features_dir, traj_dir, sampled_frames_file, clusterer_dir, msm_trajectory_filename, 
                    n_clusters, start_cluster=48, n_steps=1000)

In [ ]:
mytraj = md.load("/home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/h5_trajectories/rep_1.h5")

In [ ]:
dir(mytraj)

In [ ]:
del mytraj

In [ ]:
subset = crystal_structure.atom_slice(range(0,400))
subset.xyz
print(subset.xyz)
distances = md.compute_contacts(subset)
print(distances)


In [ ]:
distances0 = distances[0]

In [ ]:
distances0 = np.nan_to_num(distances0)

In [ ]:
distances0

In [ ]:
np.where(distances0 > 0.0)